In [1]:
import re
import sys

sys.path.append('..')
sys.path.append('../..')

In [2]:
import clingo
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score

import torch
from torch.utils.data import DataLoader

In [3]:
from dnf_layer import SemiSymbolicLayerType
from rule_learner import DNFClassifierEO, DNFClassifier
from test_common import SyntheticDataset
from utils import DeltaDelayedExponentialDecayScheduler
from dnf_post_train import (
    remove_unused_conjunctions,
    remove_disjunctions_when_empty_conjunctions,
)

In [4]:
RNG_SEED = 73

NUM_IN = 15
NUM_CONJ = 9
NUM_CLASSES = 3

SYNTH_DATA_PATH = f'synth_multiclass_data_in{NUM_IN}_conj{NUM_CONJ}_out{NUM_CLASSES}.npz'

BATCH_SIZE = 256
NUM_EPOCHS = 100

In [5]:
torch.manual_seed(RNG_SEED)
np.random.seed(RNG_SEED)

In [6]:
dnpz = np.load(SYNTH_DATA_PATH)
full_nullary = dnpz['nullary']
full_target = dnpz['target']

In [7]:
nullary_full_train, nullary_test, target_full_train, target_test = train_test_split(
    full_nullary, full_target, random_state=RNG_SEED)
nullary_train, nullary_val, target_train, target_val = train_test_split(
    nullary_full_train, target_full_train, test_size=0.2, random_state=RNG_SEED
)

train_loader = DataLoader(SyntheticDataset(nullary_train, target_train), BATCH_SIZE)
val_loader = DataLoader(SyntheticDataset(nullary_val, target_val), BATCH_SIZE)
test_loader = DataLoader(SyntheticDataset(nullary_test, target_test), BATCH_SIZE)

In [8]:
model = DNFClassifierEO(NUM_IN, NUM_CONJ, NUM_CLASSES, 0.1)
delta_decay_scheduler = DeltaDelayedExponentialDecayScheduler(
    initial_delta=0.1,
    delta_decay_delay=10,
    delta_decay_steps=1,
    delta_decay_rate=1.1,
)
optimiser = torch.optim.Adam(model.parameters(), 0.001)
criterion = torch.nn.CrossEntropyLoss()

In [9]:
for i in range(NUM_EPOCHS):
    model.train()
    epoch_loss = []
    for x, y in train_loader:
        optimiser.zero_grad()
        y_hat = model(x.float())
        loss = criterion(y_hat, y.float())
        loss.backward()
        optimiser.step()

        epoch_loss.append(loss.item())
    
    model.eval()    
    val_correct = 0
    val_total = 0
    for x, y in val_loader:
        with torch.no_grad():
            y_hat = torch.tanh(model(x.float()))
            _, y_pred = torch.max(y_hat, 1)
            val_correct += torch.sum(y_pred == (torch.argmax(y, dim=1))).item()
            val_total += len(y)

    new_delta_val = delta_decay_scheduler.step(model, i)
    avg_loss = sum(epoch_loss) / len(epoch_loss)
    print(f'[{i + 1:3d}] Delta: {new_delta_val:.3f}  '
          f'Train avg loss: {avg_loss:.3f}  '
          f'Val acc: {val_correct / val_total:.3f}')

[  1] Delta: 0.100  Train avg loss: 1.021  Val acc: 0.739
[  2] Delta: 0.100  Train avg loss: 0.642  Val acc: 0.949
[  3] Delta: 0.100  Train avg loss: 0.409  Val acc: 0.990
[  4] Delta: 0.100  Train avg loss: 0.266  Val acc: 0.997
[  5] Delta: 0.100  Train avg loss: 0.180  Val acc: 1.000
[  6] Delta: 0.100  Train avg loss: 0.129  Val acc: 1.000
[  7] Delta: 0.100  Train avg loss: 0.097  Val acc: 1.000
[  8] Delta: 0.100  Train avg loss: 0.076  Val acc: 1.000
[  9] Delta: 0.100  Train avg loss: 0.062  Val acc: 1.000
[ 10] Delta: 0.100  Train avg loss: 0.051  Val acc: 1.000
[ 11] Delta: 0.100  Train avg loss: 0.044  Val acc: 1.000
[ 12] Delta: 0.110  Train avg loss: 0.037  Val acc: 1.000
[ 13] Delta: 0.121  Train avg loss: 0.033  Val acc: 1.000
[ 14] Delta: 0.133  Train avg loss: 0.030  Val acc: 1.000
[ 15] Delta: 0.146  Train avg loss: 0.027  Val acc: 1.000
[ 16] Delta: 0.161  Train avg loss: 0.025  Val acc: 1.000
[ 17] Delta: 0.177  Train avg loss: 0.023  Val acc: 1.000
[ 18] Delta: 0

In [10]:
torch.save(model.state_dict(), f'dnfeo_multi_class_synth_{RNG_SEED}.pth')
# model.load_state_dict(torch.load(f'dnfeo_synth_{RNG_SEED}.pth')) 

In [11]:
model2 = DNFClassifier(NUM_IN, NUM_CONJ, NUM_CLASSES, 1)

sd = model.state_dict()
sd.pop('eo_layer.weights')
model2.load_state_dict(sd)

<All keys matched successfully>

In [12]:
def eval_multi_class_dnf(model, data_loader):
    # Return accuracy and Jaccard score
    correct = 0
    total = 0
    jaccard_scores = []
    for x, y in data_loader:
        with torch.no_grad():
            y_hat = torch.tanh(model(x.float()))
            _, y_pred = torch.max(y_hat, 1)
            correct += torch.sum(y_pred == (torch.argmax(y, dim=1))).item()
            total += len(y)
            y_hat_jacc = (y_hat > 0).int()
            jacc = jaccard_score(y.detach().cpu().numpy(), y_hat_jacc.detach().cpu().numpy(), average='samples')
            jaccard_scores.append(jacc)

    return (correct / total), (sum(jaccard_scores) / len(jaccard_scores))

In [13]:
acc, jacc =  eval_multi_class_dnf(model2, test_loader)
print(f'Accuarcy: {acc:.3f}')
print(f'Jaccard:  {jacc:.3f}')

Accuarcy: 1.000
Jaccard:  1.000


# Post Training Process

In [14]:
def prune_layer_weight(
    model,
    layer_type: SemiSymbolicLayerType,
    epsilon: float,
    data_loader: DataLoader,
) -> int:
    if layer_type == SemiSymbolicLayerType.CONJUNCTION:
        curr_weight = model.dnf.conjunctions.weights.data.clone()
    else:
        curr_weight = model.dnf.disjunctions.weights.data.clone()

    _, og_jacc = eval_multi_class_dnf(model, data_loader)

    prune_count = 0
    weight_device = curr_weight.device

    flatten_weight_len = len(torch.reshape(curr_weight, (-1,)))
    base_iterator = range(flatten_weight_len)
    for i in base_iterator:
        curr_weight_flatten = torch.reshape(curr_weight, (-1,))

        if curr_weight_flatten[i] == 0:
            continue

        mask = torch.ones(flatten_weight_len, device=weight_device)
        mask[i] = 0
        mask = mask.reshape(curr_weight.shape)

        masked_weight = curr_weight * mask

        if layer_type == SemiSymbolicLayerType.CONJUNCTION:
            model.dnf.conjunctions.weights.data = masked_weight
        else:
            model.dnf.disjunctions.weights.data = masked_weight

        _, new_jacc = eval_multi_class_dnf(model, data_loader)
        performance_drop = og_jacc - new_jacc
        if performance_drop < epsilon:
            prune_count += 1
            curr_weight *= mask

    if layer_type == SemiSymbolicLayerType.CONJUNCTION:
        model.dnf.conjunctions.weights.data = curr_weight
    else:
        model.dnf.disjunctions.weights.data = curr_weight
    return prune_count

In [15]:
prune_epsilon = 0.005
print('Prune disj layer')
prune_count = prune_layer_weight(model2, SemiSymbolicLayerType.DISJUNCTION, 
    prune_epsilon, val_loader)
_, new_jacc = eval_multi_class_dnf(model2, val_loader)
print(f'Pruned disj count:   {prune_count}')
print(f'New perf after disj: {new_jacc:.3f}\n')

unused_conj = remove_unused_conjunctions(model)
print(f'Remove unused conjunctions: {unused_conj}')
print()

print('Prune conj layer')
prune_count = prune_layer_weight(model2, SemiSymbolicLayerType.CONJUNCTION, 
    prune_epsilon, val_loader)
_, new_jacc = eval_multi_class_dnf(model2, val_loader)
print(f'Pruned conj count:   {prune_count}')
print(f'New perf after conj: {new_jacc:.3f}\n')

removed_disj = remove_disjunctions_when_empty_conjunctions(model)
print(
    f'Remove disjunction that uses empty conjunctions: {removed_disj}'
)
print()

print('Prune disj layer again')
prune_count = prune_layer_weight(model2, SemiSymbolicLayerType.DISJUNCTION,
    prune_epsilon, val_loader)
_, new_jacc = eval_multi_class_dnf(model2, val_loader)
print(f'Pruned disj count:   {prune_count}')
print(f'New perf after disj: {new_jacc:.3f}')

Prune disj layer
Pruned disj count:   22
New perf after disj: 1.000

Remove unused conjunctions: 0

Prune conj layer
Pruned conj count:   113
New perf after conj: 0.996

Remove disjunction that uses empty conjunctions: 0

Prune disj layer again
Pruned disj count:   1
New perf after disj: 0.996


In [16]:
sd = model.state_dict()
sd.pop('eo_layer.weights')
model2.load_state_dict(sd)
pre_threshold_sd = model2.state_dict()

In [17]:
def apply_threshold(
    model, og_conj_weight, og_disj_weight, t_val, const: float = 6.0,
) -> None:
    new_conj_weight = (
        (torch.abs(og_conj_weight) > t_val) * torch.sign(og_conj_weight) * const
    )
    model.dnf.conjunctions.weights.data = new_conj_weight

    new_disj_weight = (
        (torch.abs(og_disj_weight) > t_val) * torch.sign(og_disj_weight) * const
    )
    model.dnf.disjunctions.weights.data = new_disj_weight

In [18]:
# model2.load_state_dict(pre_threshold_sd)

conj_min = torch.min(model2.dnf.conjunctions.weights.data)
conj_max = torch.max(model2.dnf.conjunctions.weights.data)
disj_min = torch.min(model2.dnf.disjunctions.weights.data)
disj_max = torch.max(model2.dnf.disjunctions.weights.data)

abs_max = torch.max(torch.abs(torch.Tensor([conj_min, conj_max, disj_min, disj_max])))

og_conj_weight = model2.dnf.conjunctions.weights.data.clone()
og_disj_weight = model2.dnf.disjunctions.weights.data.clone()

jacc_scores = []
t_vals = torch.arange(0, abs_max + 0.01, 0.01)

for v in t_vals:
    apply_threshold(model2, og_conj_weight, og_disj_weight,
                              v, 6.0)
    _, jacc = eval_multi_class_dnf(model2, val_loader)
    jacc_scores.append(jacc)

best_jacc_score = max(jacc_scores)
best_t = t_vals[torch.argmax(torch.Tensor(jacc_scores))]
print(f'Best t: {best_t.item():.3f}    Acc: {best_jacc_score:.3f}')
apply_threshold(model2, og_conj_weight, og_disj_weight, best_t)
_, final_jacc = eval_multi_class_dnf(model2, val_loader)
print(f'Jacc after threshold: {final_jacc:.3f}')

Best t: 0.250    Acc: 0.981
Jacc after threshold: 0.981


In [19]:
final_sd = model2.state_dict()

output_rules = []

# Get all conjunctions
conj_w = final_sd["dnf.conjunctions.weights"]
conjunction_map = dict()
for i, w in enumerate(conj_w):
    if torch.all(w == 0):
        # No conjunction is applied here
        continue

    conjuncts = []
    for j, v in enumerate(w):
        if v < 0:
            # Negative weight, negate the atom
            conjuncts.append(f"not has_attr_{j}")
        elif v > 0:
            # Positive weight, normal atom
            conjuncts.append(f"has_attr_{j}")

    conjunction_map[i] = conjuncts

# Get DNF
disj_w = final_sd["dnf.disjunctions.weights"]
not_covered_classes = []
for i, w in enumerate(disj_w):
    if torch.all(w == 0):
        # No DNF for class i
        not_covered_classes.append(i)
        continue

    disjuncts = []
    for j, v in enumerate(w):
        if v < 0 and j in conjunction_map:
            # Negative weight, negate the existing conjunction
            
            # Need to add auxilary predicate (conj_X) which is not yet
            # in the final rules list
            output_rules.append(
                f"conj_{j} :- {', '.join(conjunction_map[j])}."
            )
            output_rules.append(f"class({i}) :- not conj_{j}.")
        elif v > 0 and j in conjunction_map:
            # Postivie weight, add normal conjunction
            body = ", ".join(conjunction_map[j])
            output_rules.append(f"class({i}) :- {body}.")

In [20]:
show_statements = [f'#show class/1.']


jacc_scores = []
total_sample_count = 0

for x_batch, y_batch in test_loader:
    for i in range(len(x_batch)):
        x = x_batch[i]
        y = torch.where(y_batch[i] == 1)[0].item()
        x_asp = [f"has_attr_{j}." for j in range(len(x)) if x[j] == 1]
        ctl = clingo.Control(["--warn=none"])
        ctl.add("base", [], " ".join(x_asp + output_rules + show_statements))
        ctl.ground([("base", [])])
        with ctl.solve(yield_=True) as handle:  # type: ignore
            all_answer_sets = [str(a) for a in handle]

        target_class = f"class({y})"

        if len(all_answer_sets) != 1:
            # No or multiple answer sets, should not happen
            print('No or multiple answer sets when evaluating rules.')
            continue

        output_classes = all_answer_sets[0].split(" ")
        output_classes_set = set(output_classes)

        target_class_set = {target_class}

        jacc = len(output_classes_set & target_class_set) / len(
            output_classes_set | target_class_set
        )
        jacc_scores.append(jacc)
        total_sample_count += 1

print(f'Jaccard score: {sum(jacc_scores) / total_sample_count:.3f}')

Jaccard score: 0.974


In [21]:
output_rules

['class(0) :- has_attr_0, has_attr_6, not has_attr_9, has_attr_10, has_attr_13, has_attr_14.',
 'class(0) :- has_attr_0, has_attr_6, has_attr_8, has_attr_10, has_attr_13, has_attr_14.',
 'class(1) :- has_attr_7, has_attr_9, has_attr_11.',
 'class(2) :- has_attr_3, has_attr_5, has_attr_6, has_attr_9, not has_attr_10, not has_attr_11, has_attr_13.',
 'conj_3 :- has_attr_2, has_attr_7, has_attr_8.',
 'class(2) :- not conj_3.',
 'class(2) :- not has_attr_2, not has_attr_8.',
 'class(2) :- has_attr_3, has_attr_5, not has_attr_11, has_attr_13.']

In [22]:
dnpz['rule_str']

array(['c0 :- a0, a2, a6, a7, a8, a10, a13, a14.',
       'c1 :- a1, a2, a4, a9, a12, a13.',
       'c2 :- a2, a3, a4, a7, a11, a12, a14.',
       'c3 :- a1, a4, a6, a11, a12, a13.', 'c4 :- a2, a7, a8, a9, a11.',
       'c5 :- a1, a2, a4, a6, a8, a11, a13.',
       'c6 :- a2, a3, a5, a6, a8, a9, a13.',
       'c7 :- a3, a6, a9, a10, a11, a12.', 'c8 :- a5, a9, a10.',
       'class_0 :- c0.', 'class_1 :- c4.', 'class_1 :- c5.',
       'class_2 :- c6.', 'class_2 :- c8.'], dtype='<U40')